In [7]:
model = "MESSAGEix-Materials"
scenario = "baseline_DEFAULT_test_0310_macro"


import ixmp
import message_ix
mp = ixmp.Platform("ixmp_dev")
scen = message_ix.Scenario(mp, model, scenario)

<IPython.core.display.Javascript object>

In [2]:
model = "MESSAGEix-Materials"
scenario = "baseline_DEFAULT_0108_macro"


import ixmp
import message_ix
mp = ixmp.Platform("ixmp_dev")
scen = message_ix.Scenario(mp, model, scenario)

In [7]:
scen.to_excel("C:/Users\maczek\PycharmProjects\message_data/macro_dump.xlsx")

In [8]:
scen.has_solution()

True

In [9]:
base = scen.clone(model=scen.model, scenario="SSP_supply_cost_test_baseline_macro", keep_solution=False)

In [4]:
from message_ix_models.tools.costs.config import Config
from message_ix_models.tools.costs.projections import create_cost_projections

In [6]:
cfg = Config(module="materials", ref_region="R12_NAM", method="convergence", format="message", scenario="SSP2")

out_materials = create_cost_projections(
    node=cfg.node,
    ref_region=cfg.ref_region,
    base_year=cfg.base_year,
    module=cfg.module,
    method=cfg.method,
    scenario_version=cfg.scenario_version,
    scenario=cfg.scenario,
    convergence_year=cfg.convergence_year,
    fom_rate=cfg.fom_rate,
    format=cfg.format,
)

Selected module: materials
Selected node: R12
Selected reference region: R12_NAM
Selected base year: 2021
Selected method: convergence
Selected fixed O&M rate: 0.025
Selected format: message
Selected scenario: SSP2
Selected convergence year: 2050
For the convergence method, only the SSP scenario(s) itself         needs to be specified.         No scenario version (previous vs. updated) is needed.
...Calculating regional differentiation in base year+region...
The following technologies are not projected due to insufficient data:
CH2O_synth
CH2O_to_resin
DUMMY_alumina_supply
DUMMY_coal_supply
DUMMY_gas_supply
DUMMY_limestone_supply_cement
DUMMY_limestone_supply_steel
DUMMY_ore_supply
agg_ref
feedstock_t/d
finishing_aluminum
gas_processing_petro
hydrotreating_ref
import_NFert
import_NH3
import_aluminum
import_petro
import_steel
manuf_aluminum
meth_bal
meth_imp
meth_t_d
meth_t_d_material
meth_trd
other_EOL_aluminum
other_EOL_cement
other_EOL_steel
prep_secondary_aluminum_1
prep_secondary_a

,node_loc,technology,year_vtg,year_act,value,unit
0,R12_AFR,bio_hpl,1960,1960,9.519231,USD/kWa
1,R12_AFR,bio_hpl,1960,1965,9.519231,USD/kWa
2,R12_AFR,bio_hpl,1960,1970,9.519231,USD/kWa
3,R12_AFR,bio_hpl,1960,1975,9.519231,USD/kWa
4,R12_AFR,bio_hpl,1960,1980,9.519231,USD/kWa
...,...,...,...,...,...,...
11799575,R12_WEU,visbreaker_ref,2090,2100,1.478498,USD/kWa
11799576,R12_WEU,visbreaker_ref,2090,2110,1.892602,USD/kWa
11799577,R12_WEU,visbreaker_ref,2100,2100,1.155000,USD/kWa
11799578,R12_WEU,visbreaker_ref,2100,2110,1.478498,USD/kWa


In [11]:
base.check_out()
base.add_par("fix_cost", )

ValueError: The index set 'technology' does not have an element 'meth_i'!

In [13]:
tec_set = list(base.set("technology"))

In [14]:
fix_cost = out_materials.fix_cost.drop_duplicates().drop(["scenario_version", "scenario"], axis=1)

In [17]:
base.add_par("fix_cost", fix_cost[fix_cost["technology"].isin(tec_set)])

In [18]:
base.commit("update cost assumption with tool")

In [ ]:
def modify_costs_with_tool(scen_name, ssp):
    mp = ixmp.Platform("ixmp_dev")
    base = message_ix.Scenario(mp, "SSP_supply_cost_test_baseline", scenario=scen_name)
    scen = base.clone(model=base.model, scenario=base.scenario.replace("baseline", ssp))
    
    tec_set = list(scen.set("technology"))
    
    cfg = Config(module="materials", ref_region="R12_NAM", method="convergence", format="message", scenario=ssp)
    
    out_materials = create_cost_projections(
        node=cfg.node,
        ref_region=cfg.ref_region,
        base_year=cfg.base_year,
        module=cfg.module,
        method=cfg.method,
        scenario_version=cfg.scenario_version,
        scenario=cfg.scenario,
        convergence_year=cfg.convergence_year,
        fom_rate=cfg.fom_rate,
        format=cfg.format,
    )
    
    fix_cost = out_materials.fix_cost.drop_duplicates().drop(["scenario_version", "scenario"], axis=1)
    scen.add_par("fix_cost", fix_cost[fix_cost["technology"].isin(tec_set)])
    inv_cost = out_materials.inv_cost.drop_duplicates().drop(["scenario_version", "scenario"], axis=1)
    scen.add_par("fix_cost", inv_cost[inv_cost["technology"].isin(tec_set)])
    
    scen.commit("update cost assumption with tool")
    #scen.solve(model="MESSAGE")

In [22]:
mp.scenario_list(model="MESSAGEix-Materials")

,model,scenario,scheme,is_default,is_locked,cre_user,cre_date,upd_user,upd_date,lock_user,lock_date,annotation,version
0,MESSAGEix-Materials,2.0deg_no_SDGs_petro_thesis_1_final_macro,MESSAGE,1,0,maczek,2023-06-27 23:24:23.000000,maczek,2023-07-03 17:01:57.000000,None,None,clone Scenario from 'MESSAGEix-Materials|NoPol...,1
1,MESSAGEix-Materials,2.0deg_no_SDGs_petro_thesis_1_macro,MESSAGE,1,0,maczek,2023-05-09 09:34:33.000000,maczek,2023-05-09 10:28:12.000000,None,None,clone Scenario from 'MESSAGEix-Materials|NoPol...,4
2,MESSAGEix-Materials,2.0deg_petro_thesis_1,MESSAGE,1,0,maczek,2023-02-08 23:38:03.000000,maczek,2023-02-08 23:44:53.000000,None,None,clone Scenario from 'MESSAGEix-Materials|NoPol...,1
3,MESSAGEix-Materials,2.0deg_petro_thesis_2,MESSAGE,1,0,maczek,2023-04-06 16:44:33.000000,None,None,None,None,clone Scenario from 'MESSAGEix-Materials|NoPol...,6
4,MESSAGEix-Materials,2.0deg_petro_thesis_2_final_macro,MESSAGE,1,0,maczek,2023-06-21 23:48:48.000000,maczek,2023-06-22 00:36:08.000000,None,None,clone Scenario from 'MESSAGEix-Materials|NoPol...,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
366,MESSAGEix-Materials,test_ccs,MESSAGE,1,0,unlu,2022-07-12 15:32:10.000000,None,None,None,None,clone Scenario from 'MESSAGEix-Materials|NoPol...,1
367,MESSAGEix-Materials,test_foil_trp_constraint,MESSAGE,1,0,maczek,2023-10-12 16:37:45.000000,None,None,None,None,clone Scenario from 'MESSAGEix-Materials|NoPol...,1
368,MESSAGEix-Materials,test_foil_trp_constraint_IEA_calib,MESSAGE,1,0,maczek,2023-10-16 13:51:53.000000,None,None,None,None,clone Scenario from 'MESSAGEix-Materials|test_...,1
369,MESSAGEix-Materials,test_report_JM,MESSAGE,1,1,min,2021-08-02 16:17:16.000000,None,None,min,2021-08-02 16:39:25.000000,"clone Scenario from 'Buildings_SSP2|test', ver...",1


In [77]:
import pandas as pd

In [78]:
r_df = pd.read_csv("C:/Users\maczek\PycharmProjects\message_data\message_data\model\material/output_file.csv")
py_df = pd.read_csv("C:/Users\maczek\PycharmProjects\message_data\message_data\model\material/py_output.csv")#, usecols=[1,2,3])

In [79]:
py_df

,Unnamed: 0,region,reg_no,year,consumption,pop,gdp_pcap,cons_pcap,del_t
0,0,Canada,1,1970,11085,21.71685,17450.805611,510.433143,-40
1,1,USA,2,1970,127304,210.11660,20233.017350,605.873120,-40
2,2,Mexico,3,1970,4168,50.59590,6910.351162,82.378216,-40
3,3,Rest Central America,4,1970,434,41.86008,5032.145662,10.367873,-40
4,4,Brazil,5,1970,6088,95.98846,4246.812446,63.424291,-40
...,...,...,...,...,...,...,...,...,...
1059,1113,Indonesia,22,2012,15006,244.18970,4272.029000,61.452223,2
1060,1114,Japan,23,2012,68800,126.60790,31957.190000,543.410008,2
1061,1115,Oceania,24,2012,7945,29.71791,30264.620000,267.347199,2
1062,1116,Other South Asia,25,2012,5959,445.52510,2062.911000,13.375228,2


In [88]:
df = r_df.set_index(["reg_no", "year"]).join(py_df.set_index(["reg_no", "year"])["gdp_pcap"])

In [85]:
r_df

,Unnamed: 0,reg_no,region,year,consumption,pop,gdp.pcap,cons.pcap,del.t
0,1,1,Canada (1),1970,11085,21.71685,17450.805611,510.433143,-40
1,2,1,Canada (1),1971,11753,22.04843,18025.700000,533.053827,-39
2,3,1,Canada (1),1972,12842,22.34410,18755.830000,574.737850,-38
3,4,1,Canada (1),1973,14153,22.61467,19822.000000,625.832701,-37
4,5,1,Canada (1),1974,15458,22.87666,20318.250000,675.710528,-36
...,...,...,...,...,...,...,...,...,...
1059,1060,26,Other South Africa (26),2008,272,130.25150,1871.848000,2.088268,-2
1060,1061,26,Other South Africa (26),2009,344,133.53860,1843.252000,2.576034,-1
1061,1062,26,Other South Africa (26),2010,355,136.94900,1909.069000,2.592206,0
1062,1063,26,Other South Africa (26),2011,474,140.48880,1988.006000,3.373934,1


In [90]:
import numpy as np
df[np.isclose(df["gdp.pcap"], df["gdp_pcap"])]

Unnamed: 0                   region  consumption        pop  \
reg_no year                                                                
1      1970           1               Canada (1)        11085   21.71685   
       1971           2               Canada (1)        11753   22.04843   
       1972           3               Canada (1)        12842   22.34410   
       1973           4               Canada (1)        14153   22.61467   
       1974           5               Canada (1)        15458   22.87666   
...                 ...                      ...          ...        ...   
26     2008        1060  Other South Africa (26)          272  130.25150   
       2009        1061  Other South Africa (26)          344  133.53860   
       2010        1062  Other South Africa (26)          355  136.94900   
       2011        1063  Other South Africa (26)          474  140.48880   
       2012        1064  Other South Africa (26)          439  144.14590   

                 gdp.pcap   cons.pcap  del.t      gdp_pcap  
reg_no year                                                 
1      1970  17450.805611  510.433143    -40  17450.805611  
       1971  18025.700000  533.053827    -39  18025.700000  
       1972  18755.830000  574.737850    -38  18755.830000  
       1973  19822.000000  625.832701    -37  19822.000000  
       1974  20318.250000  675.710528    -36  20318.250000  
...                   ...         ...    ...           ...  
26     2008   1871.848000    2.088268     -2   1871.848000  
       2009   1843.252000    2.576034     -1   1843.252000  
       2010   1909.069000    2.592206      0   1909.069000  
       2011   1988.006000    3.373934      1   1988.006000  
       2012   2063.026000    3.045525      2   2063.026000  

[1064 rows x 8 columns]

In [91]:
r_df

,Unnamed: 0,reg_no,region,year,consumption,pop,gdp.pcap,cons.pcap,del.t
0,1,1,Canada (1),1970,11085,21.71685,17450.805611,510.433143,-40
1,2,1,Canada (1),1971,11753,22.04843,18025.700000,533.053827,-39
2,3,1,Canada (1),1972,12842,22.34410,18755.830000,574.737850,-38
3,4,1,Canada (1),1973,14153,22.61467,19822.000000,625.832701,-37
4,5,1,Canada (1),1974,15458,22.87666,20318.250000,675.710528,-36
...,...,...,...,...,...,...,...,...,...
1059,1060,26,Other South Africa (26),2008,272,130.25150,1871.848000,2.088268,-2
1060,1061,26,Other South Africa (26),2009,344,133.53860,1843.252000,2.576034,-1
1061,1062,26,Other South Africa (26),2010,355,136.94900,1909.069000,2.592206,0
1062,1063,26,Other South Africa (26),2011,474,140.48880,1988.006000,3.373934,1


In [51]:
py_df#["region"].unique()

,region,year,consumption
0,Canada,1970,11085
1,USA,1970,127304
2,Mexico,1970,4168
3,Rest Central America,1970,434
4,Brazil,1970,6088
...,...,...,...
866,Korea,2012,57650
867,Japan,2012,68800
868,Oceania,2012,7945
869,Other South Asia,2012,5959


In [54]:
py_df

,Unnamed: 0,region,year,consumption,pop,gdp_pcap,cons_pcap,del_t
0,0,Canada,1970,11085,21.71685,17450.805611,510.433143,-40
1,1,USA,1970,127304,210.11660,20233.017350,605.873120,-40
2,2,Mexico,1970,4168,50.59590,6910.351162,82.378216,-40
3,3,Rest Central America,1970,434,41.86008,5032.145662,10.367873,-40
4,4,Brazil,1970,6088,95.98846,4246.812446,63.424291,-40
...,...,...,...,...,...,...,...,...
866,898,Korea,2012,57650,72.94119,19790.770000,790.362757,2
867,899,Japan,2012,68800,126.60790,31957.190000,543.410008,2
868,900,Oceania,2012,7945,29.71791,30264.620000,267.347199,2
869,901,Other South Asia,2012,5959,445.52510,2062.911000,13.375228,2


In [67]:
r_df["region"] = r_df["region"].str.replace("(","").str.replace(")","").str.replace('\d+', '').str[:-1]

C:\Users\maczek\AppData\Local\Temp\ipykernel_5280\2693521517.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  r_df["region"] = r_df["region"].str.replace("(","").str.replace(")","").str.replace('\d+', '').str[:-1]
C:\Users\maczek\AppData\Local\Temp\ipykernel_5280\2693521517.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  r_df["region"] = r_df["region"].str.replace("(","").str.replace(")","").str.replace('\d+', '').str[:-1]


In [4]:
import pandas as pd
import yaml


path = "C:/Users\maczek\PycharmProjects\message_data\data\material/aluminum/demand_alu.yaml"
path = "C:/Users\maczek\PycharmProjects\message_data\data\material/steel_cement/demand_cement.yaml"
path = "C:/Users\maczek\PycharmProjects\message_data\data\material/steel_cement/demand_steel.yaml"

with open(path, 'r') as file:
    yaml_data = file.read()

data_list = yaml.safe_load(yaml_data)

# Convert to DataFrame
df = pd.DataFrame([(key, value['year'], value['value']) for entry in data_list for key, value in entry.items()],
                  columns=['Region', 'Year', 'Value'])